In [15]:
from pynq import Overlay, GPIO, Register, allocate, MMIO
import os
import struct
import numpy as np

def float_to_int(f):
    return (struct.unpack('<I', struct.pack('<f', f))[0])

def int_to_float(i):
    return (struct.unpack('<f', struct.pack('<I', i))[0])
    


In [16]:
overlay = Overlay("nn.bit")
fcc1=overlay.backward_fcc_0
fcc1.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=0),
  w = Register(w=0),
  b = Register(b=0),
  dx = Register(dx=0),
  dy = Register(dy=0),
  xdimension = Register(xdimension=0),
  ydimension = Register(ydimension=0),
  lr = Register(lr=0)
}

In [17]:
BASE_ADDRESS_X=0x4001_0000
BASE_ADDRESS_W=0x4001_0100
BASE_ADDRESS_Y=0x4001_0200
BASE_ADDRESS_B=0x4001_0300
BASE_ADDRESS_DX=0x4001_0400
BASE_ADDRESS_DY=0x4001_0500



ADDRESS_RANGE=0xF0

mmio_x= MMIO(BASE_ADDRESS_X,ADDRESS_RANGE)
mmio_w= MMIO(BASE_ADDRESS_W,ADDRESS_RANGE)
mmio_y= MMIO(BASE_ADDRESS_Y,ADDRESS_RANGE)
mmio_b= MMIO(BASE_ADDRESS_B,ADDRESS_RANGE)
mmio_dx= MMIO(BASE_ADDRESS_DX,ADDRESS_RANGE)
mmio_dy= MMIO(BASE_ADDRESS_DY,ADDRESS_RANGE)

OFFSET_X=0x0
OFFSET_Y=0x0
OFFSET_W=0x0
OFFSET_B=0x0


xdim=3
ydim=1

In [18]:
for i in range(xdim):
    mmio_x.write(i*4,float_to_int(i*0.5))
    mmio_dx.write(i*4,float_to_int(0))

OFFSET_W=0x0
for i in range(ydim):
    for j in range(xdim):
        mmio_w.write(OFFSET_W,float_to_int((j)))
        OFFSET_W += 4

for i in range(ydim):
    mmio_y.write(i*4,float_to_int(0))
    mmio_b.write(i*4,float_to_int(2.5))

    
mmio_dy.write(0,float_to_int(1.5))

In [19]:


fcc1.register_map.x=BASE_ADDRESS_X
fcc1.register_map.w=BASE_ADDRESS_W
fcc1.register_map.b=BASE_ADDRESS_B
fcc1.register_map.xdimension=xdim
fcc1.register_map.ydimension=ydim
fcc1.register_map.dx=BASE_ADDRESS_DX
fcc1.register_map.dy=BASE_ADDRESS_DY
fcc1.register_map.lr=float_to_int(0.1)

fcc1.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  x = Register(x=1073807360),
  w = Register(w=1073807616),
  b = Register(b=1073808128),
  dx = Register(dx=1073808384),
  dy = Register(dy=1073808640),
  xdimension = Register(xdimension=3),
  ydimension = Register(ydimension=1),
  lr = Register(lr=1036831949)
}

In [20]:
OFFSET_X=0x0
OFFSET_W=0x0
OFFSET_B=0x0

x=[]
for i in range(xdim):
    x.append(int_to_float(mmio_x.read(OFFSET_X)))
    OFFSET_X= OFFSET_X+4


w=[]
for i in range(xdim*ydim):
    w.append(int_to_float(mmio_w.read(OFFSET_W)))
    OFFSET_W= OFFSET_W+4
    

OFFSET_DX=0x0
dx=[]
for i in range(xdim):
    dx.append(int_to_float(mmio_dx.read(OFFSET_DX)))
    OFFSET_DX+=4
    
OFFSET_DY=0x0
dy=[]
for i in range(ydim):
    dy.append(int_to_float(mmio_dy.read(OFFSET_DY)))
    OFFSET_DY+=4
    
OFFSET_B=0x0
b=[]
for i in range(ydim):
    b.append(int_to_float(mmio_b.read(OFFSET_B)))
    OFFSET_B+=4
    
print("x= ",x)
print("w= ",w)
print("b= ",b)
print("dx= ",dx)
print("dy= ",dy)

x=  [0.0, 0.5, 1.0]
w=  [0.0, 1.0, 2.0]
b=  [2.5]
dx=  [0.0, 0.0, 0.0]
dy=  [1.5]


In [21]:
fcc1.write(0x00, 1)
fpga_state = fcc1.read(0x00)

max_try = 1000000
while fpga_state != 6 and fpga_state != 4:
    fpga_state = fcc1.read(0x00)
    max_try = max_try -1
    if max_try == 0:
        print("ERROR: Can't go ahead")
        fcc1.write(0x00, 4)
        break

fcc1.write(0x00, 4)
# print(xbuffer)
# print(wbuffer)
# print(ybuffer)
# y[:ydim] = ybuffer[:ydim]

In [22]:

OFFSET_X=0x0
OFFSET_W=0x0
OFFSET_B=0x0

x=[]
for i in range(xdim):
    x.append(int_to_float(mmio_x.read(OFFSET_X)))
    OFFSET_X= OFFSET_X+4


w=[]
for i in range(xdim*ydim):
    w.append(int_to_float(mmio_w.read(OFFSET_W)))
    OFFSET_W= OFFSET_W+4
    

OFFSET_DX=0x0
dx=[]
for i in range(xdim):
    dx.append(int_to_float(mmio_dx.read(OFFSET_DX)))
    OFFSET_DX+=4
    
OFFSET_DY=0x0
dy=[]
for i in range(ydim):
    dy.append(int_to_float(mmio_dy.read(OFFSET_DY)))
    OFFSET_DY+=4
    
OFFSET_B=0x0
b=[]
for i in range(ydim):
    b.append(int_to_float(mmio_b.read(OFFSET_B)))
    OFFSET_B+=4
    
print("x= ",x)
print("w= ",w)
print("b= ",b)
print("dx= ",dx)
print("dy= ",dy)

x=  [0.0, 0.5, 1.0]
w=  [0.0, 0.925000011920929, 1.850000023841858]
b=  [2.3499999046325684]
dx=  [0.0, 1.5, 3.0]
dy=  [1.5]
